## Device Exposure Table Mapping

This is an attempt at mapping FHIR to OMOP using the following guide: https://build.fhir.org/ig/HL7/cdmh/profiles.html#omop-to-fhir-mappings
<br>In this notebook we are mapping FHIR to the OMOP Device Exposure Table

### Load Data Frame from Parquet Catalog File

In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import dayofmonth,month,year,to_date,trunc,split,explode,array,col

# Create a local Spark session
spark = SparkSession.builder.appName('etl').getOrCreate()

In [23]:
# Reads file 
df = spark.read.parquet('data/catalog.parquet')

Data frame schema 

In [24]:
#df.printSchema()

### Procedure Mapping 

Filter By procedure resource type 

In [25]:
filtered = df.filter(df['resourceType'] == 'Procedure')

In [26]:
#filtered.printSchema()

Selects relevant fields. Using alias so we don't have columns with the same name

In [39]:
Procedure = filtered.select("id",
  col("encounter.reference").alias("visit_occurrence_id"),
  col("subject.reference").alias("person_id"),
  "performedPeriod",
  #"focalDevice",
  "performedPeriod",
  "performer",
  "extension.valueCodeableConcept")

Extract the start date and time from the period field.

In [35]:
#splits the date and time
split_start = split(Procedure['performedPeriod.start'], 'T')
split_end = split(Procedure['performedPeriod.end'], 'T')

#assigns each to a column 
procedure_date_and_time = Procedure\
    .withColumn("device_exposure_start_date",split_start.getItem(0))\
    .withColumn("device_exposure_start_datetime",split_start.getItem(1))\
    .withColumn("device_exposure_end_date",split_end.getItem(0))\
    .withColumn("device_exposure_end_datetime",split_end.getItem(1))


Drop columns no longer needed

In [40]:
dropped  = procedure_date_and_time.drop("performedPeriod")

Rename the columns 

In [41]:
device_exposure = dropped\
    .withColumnRenamed("id","device_exposure_id")\
    .withColumnRenamed("valueCodeableConcept","device_type_concept_id")\
    .withColumnRenamed("performer","provider_id")

Shows mapped output table

In [42]:
device_exposure.show(5) 

+--------------------+--------------------+--------------------+-----------+------+--------------------------+------------------------------+------------------------+----------------------------+
|  device_exposure_id| visit_occurrence_id|           person_id|provider_id|coding|device_exposure_start_date|device_exposure_start_datetime|device_exposure_end_date|device_exposure_end_datetime|
+--------------------+--------------------+--------------------+-----------+------+--------------------------+------------------------------+------------------------+----------------------------+
|0ab70f34-43d4-4ae...|urn:uuid:fc7e46e8...|urn:uuid:46c99701...|       null|  null|                2011-05-04|                23:11:57+03:00|              2011-05-04|              23:30:57+03:00|
|856dafba-7be8-461...|urn:uuid:cc07345b...|urn:uuid:46c99701...|       null|  null|                2011-07-07|                00:32:57+03:00|              2011-07-07|              00:55:57+03:00|
|7027cb03-ed07-41c..